# Цель

Для экспериментов см в experiments / genetic_init.py


- Валидация графа (WG) в utilities / validation 


In [36]:
from sampo.generator import SimpleSynthetic
from sampo.schemas.graph import WorkGraph
import numpy as np 
from sampo.schemas.schedule_spec import ScheduleSpec

ChromosomeType = tuple[np.ndarray, np.ndarray, np.ndarray, ScheduleSpec, np.ndarray]

N = 35
wg = WorkGraph.loadf(f'wgs/{N}', f'{N}_0')
ss = SimpleSynthetic(rand=1)
contractors = [ss.contractor(x) for x in range(5, 15 + 1)]


In [37]:
from utils import contractor
contractor()

[Contractor(id='cd613e30-d8f1-6adf-91b7-584a2265b1f5', name=None, workers={'driver': 30 driver, 'fitter': 29 fitter, 'handyman': 38 handyman, 'electrician': 10 electrician, 'manager': 14 manager, 'engineer': 9 engineer}, equipments={}), Contractor(id='025b413f-8a9a-021e-a648-a7dd06839eb9', name=None, workers={'driver': 38 driver, 'fitter': 35 fitter, 'handyman': 49 handyman, 'electrician': 13 electrician, 'manager': 19 manager, 'engineer': 12 engineer}, equipments={}), Contractor(id='4be03db0-dc25-74bd-b940-67edfe175330', name=None, workers={'driver': 43 driver, 'fitter': 43 fitter, 'handyman': 55 handyman, 'electrician': 13 electrician, 'manager': 22 manager, 'engineer': 14 engineer}, equipments={}), Contractor(id='e1fab9d7-8c7e-134f-5dfb-d3d12c4a3698', name=None, workers={'driver': 47 driver, 'fitter': 45 fitter, 'handyman': 62 handyman, 'electrician': 17 electrician, 'manager': 22 manager, 'engineer': 16 engineer}, equipments={}), Contractor(id='eb8ac8ce-8a24-5e6b-3313-8131c541013d'

In [40]:
wg.to_frame().head()

,activity_id,activity_name,model_name,volume,priority,predecessor_ids,connection_types,lags
0,32435d5b-66d1-4955-8341-efadaaa9af6b,minimal road,"{'granular_name': 'minimal road', 'measurement...",7.055065,1,,,
1,52670047-43d8-4d29-a182-acb2b03d82ba,temporary road,"{'granular_name': 'temporary road', 'measureme...",7.055065,1,32435d5b-66d1-4955-8341-efadaaa9af6b,FFS,0.05
2,5a78b147-e3e1-43a6-8d86-d9758b5d6f91,engineering preparation,"{'granular_name': 'engineering preparation', '...",0.560000,1,32435d5b-66d1-4955-8341-efadaaa9af6b,FS,0
3,199bbe9b-e9b5-473d-baf2-cf75928105ed,final road,"{'granular_name': 'final road', 'measurement':...",7.055065,1,52670047-43d8-4d29-a182-acb2b03d82ba,FFS,0.05
4,d325bd2d-b511-4653-b21e-41d25cea08be,power line,"{'granular_name': 'power line', 'measurement':...",2.762447,1,"52670047-43d8-4d29-a182-acb2b03d82ba,5a78b147-...","FS,FS","0,0"


In [42]:
from sampo.scheduler.genetic.base import GeneticScheduler
from sampo.schemas.time_estimator import DefaultWorkEstimator


init_popul = GeneticScheduler()\
            .generate_first_population(wg, contractors=contractors,   work_estimator=DefaultWorkEstimator())

init_popul.keys()

dict_keys(['lft', 'heft_end', 'heft_between', '12.5%', '25%', '75%', '87.5%'])

In [44]:
init_popul['lft'][0]

In [50]:
init_chromosomes['lft'][0]

(array([ 0,  1,  2,  3,  5, 12, 15, 16, 18, 19, 22, 24,  6,  4,  7,  8,  9,
        10, 11, 13, 14, 17, 20, 21, 23, 25, 26, 27, 28, 29, 30, 31]),
 array([[ 0,  0,  0,  0,  0,  0,  0],
        [37,  0, 51,  0, 17,  0,  1],
        [44,  0, 53,  0, 21,  0,  2],
        [30,  0, 30,  0,  6,  6,  3],
        [ 7,  7,  7,  5,  5,  0,  4],
        [ 6,  9, 13,  0,  2,  4,  5],
        [ 6,  6,  6,  4,  4,  0,  6],
        [63,  0, 95,  0, 21,  0,  7],
        [ 6,  6,  6,  3,  3,  0,  8],
        [ 6,  8, 12,  6,  8,  0,  9],
        [63, 63, 63, 31, 42,  0, 10],
        [ 4,  8, 12,  1,  4,  0,  0],
        [ 7, 10, 13,  0,  2,  4,  1],
        [ 3,  3,  3,  2,  2,  0,  2],
        [ 6,  8, 12,  6,  8,  0,  3],
        [ 8,  0,  9,  0,  2,  0,  4],
        [ 6,  9, 12,  0,  2,  4,  5],
        [25, 25, 25, 17, 17,  0,  6],
        [ 6,  9, 12,  4,  2,  0,  7],
        [ 8,  0,  9,  0,  2,  0,  8],
        [ 3,  3,  3,  2,  2,  0,  9],
        [ 3,  3,  3,  2,  2,  0, 10],
        [ 5,  8, 1

In [5]:
from sampo.schemas.schedule_spec import ScheduleSpec

schedule = init_popul['87.5%'][0]

spec = ScheduleSpec()
from sampo.utilities.validation import validate_schedule

validate_schedule(schedule, wg, contractors, spec)

In [6]:

from sampo.scheduler.genetic.converter import convert_schedule_to_chromosome, convert_chromosome_to_schedule

from sampo.scheduler.genetic.utils import prepare_optimized_data_structures
from sampo.schemas.landscape import LandscapeConfiguration

landscape = LandscapeConfiguration()

worker_pool, index2node, index2zone, work_id2index, worker_name2index, index2contractor_obj, \
        worker_pool_indices, contractor2index, contractor_borders, node_indices, priorities, parents, children, \
        resources_border, contractors_available = prepare_optimized_data_structures(wg, contractors, landscape, spec)

init_chromosomes = \
        {name: (convert_schedule_to_chromosome(work_id2index, worker_name2index,
                                               contractor2index, contractor_borders, schedule, chromosome_spec,
                                               landscape, order),
                importance, chromosome_spec)
         if schedule is not None else None
         for name, (schedule, order, chromosome_spec, importance) in init_popul.items()}

# 0 - Chromosome, 1 - Importance, 2 - Specification

In [7]:
children[3]

{4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24}

In [8]:
len(node_indices)

32

In [9]:
init_chromosomes.keys()

dict_keys(['lft', 'heft_end', 'heft_between', '12.5%', '25%', '75%', '87.5%'])

In [10]:
from sampo.scheduler.genetic.operators import TimeFitness, FitnessFunction
from sampo.base import SAMPO
for i in [0, 1, 2, 4]:
    print(f'{i} element of chromosome, np.array - size', init_chromosomes['lft'][0][i].shape)


fitness_f = TimeFitness()


# order_chromosome, resource_chromosome, resource_border_chromosome, copy.deepcopy(spec), zone_changes_chromosome

0 element of chromosome, np.array - size (32,)
1 element of chromosome, np.array - size (32, 7)
2 element of chromosome, np.array - size (11, 6)
4 element of chromosome, np.array - size (32, 0)


In [11]:
# from sampo.scheduler.genetic.operators import TimeFitness, FitnessFunction

# fitness_f = FitnessFunction()
# [fitness_f.evaluate(chromosome, TimeFitness()) for chromosome in init_chromosomes]



In [12]:
print(init_popul['lft'])

(<sampo.schemas.schedule.Schedule object at 0x12f972ec0>, [53cb089e-145c-4a41-acc6-a3ce5c467896, 32435d5b-66d1-4955-8341-efadaaa9af6b, 52670047-43d8-4d29-a182-acb2b03d82ba, 5a78b147-e3e1-43a6-8d86-d9758b5d6f91, 0e810119-6d50-4dec-ba7a-dd8bbf1ec0f1, 38e3ea7b-2352-402f-b13b-acdbdc65f25b, 5a71f079-bbf7-43d7-aa10-419af0e87980, 5c628ee4-5530-482a-b432-3577deb86895, 70c28f60-8a10-4c90-9c81-ef3af88167c2, 76b2d072-d93a-45e7-9e74-3a3983061a8a, bf9ca29b-7f43-4b6e-812e-a372bbea1758, dadaf553-2207-4959-8317-cdde704787c2, 135d2878-8311-4f40-afb5-a1072a1e49c1, 06ef9672-5368-46d2-a59f-5e85a54d516b, 199bbe9b-e9b5-473d-baf2-cf75928105ed, 222b790c-d86f-49dd-ae93-e41d337127e7, 24929731-0589-42da-8345-ee75aed8a370, 28dbef32-d5b1-44a2-a0ea-902c59fd03ed, 302f14c3-1f6d-4a14-aef3-2e9c48acfc3d, 4f101709-b26d-4198-aed5-71fff827463d, 59e9b0ff-38a0-4db8-9fce-49586013e234, 670b0dbc-5a72-49cc-8b41-f3bff386afb9, 76e5006c-aa46-4f9d-8833-60a9a67c09b2, a0f05371-6f45-4d27-b06d-d1c298c8870f, d325bd2d-b511-4653-b21e-41d25

In [26]:
def array_to_string(arr, line_separator = '\n', column_separator=' ', with_brackets=False):
    result_lines = []
    for row in arr:
        # Форматируем каждое число в строке
        formatted_row = []
        for val in row:
            if np.isnan(val):
                formatted_row.append("nan")
            elif np.isinf(val):
                formatted_row.append("inf" if val > 0 else "-inf")
            else:
                if isinstance(val, (int, np.integer)):
                    formatted_row.append(str(int(val)))
        
        # Собираем строку
        row_str = column_separator.join(formatted_row)
        
        if with_brackets:
            row_str = f"[{row_str}]"
        
        result_lines.append(row_str)
    
    return line_separator.join(result_lines)

def input_chromosome_in_text(a : ChromosomeType):
    a1, a2, a3, _, _ = a
    task_activity = '<TA>'+ ','.join([str(e) for e in a1.tolist()]) + '<\TA>\n'
    resources_for_activity = '<RA>' + array_to_string(a2) + '<\RA>'
    resources_border = '<R>' + array_to_string(a3) + '<\R>'
    return task_activity, resources_for_activity, resources_border

a, res, b = input_chromosome_in_text(init_chromosomes['lft'][0])

In [29]:
b

'<R>30 30 40 10 15 9\n37 38 51 13 17 12\n44 39 53 13 23 14\n46 49 65 17 25 17\n51 60 68 18 26 17\n62 55 85 20 28 19\n61 71 89 23 33 22\n68 73 95 26 32 24\n79 79 110 26 42 26\n88 83 113 29 42 31\n82 89 119 31 44 29<\\R>'

In [14]:
print(array_to_string(init_chromosomes['heft_end'][0][2]))

30 30 40 10 15 9
37 38 51 13 17 12
44 39 53 13 23 14
46 49 65 17 25 17
51 60 68 18 26 17
62 55 85 20 28 19
61 71 89 23 33 22
68 73 95 26 32 24
79 79 110 26 42 26
88 83 113 29 42 31
82 89 119 31 44 29


In [15]:
ta, res2 = input_chromosome_in_text(init_chromosomes['heft_end'][0])

In [16]:
task_gen = '<TA>0,1,2,3,7,5,12,15,16,18,19,22,24,6,9,4,10,8,14,11,13,17,20,21,23,25,26,27,28,29,30,31</TA>'
TA = list(map(int, task_gen[4:-5].split(',')))

In [17]:
def str_to_numpy_array(data_str):
    return np.array([[int(x) for x in line.split()] 
                     for line in data_str.strip().split('\n') 
                     if line.strip()])
RA_s='''0 0 0 0 0 0 0
39 0 46 0 19 0 1
42 0 57 0 21 0 2
30 0 41 0 16 6 3
7 7 7 5 5 0 4
6 9 13 0 2 4 5
6 6 6 4 4 0 6
78 0 100 0 34 0 7
6 6 6 3 3 0 8
6 8 12 6 8 0 9
63 60 82 30 32 0 10
4 8 12 1 4 0 0
7 10 13 0 2 4 1
3 3 3 2 2 0 2
6 8 12 6 8 0 3
8 0 9 0 2 0 4
6 9 12 0 2 4 5
25 25 25 17 17 0 6
6 9 12 4 2 0 7
8 0 9 0 2 0 8
3 3 3 2 2 0 9
3 3 3 2 2 0 10
5 8 11 0 2 4 0
25 25 25 12 17 0 1
6 9 12 0 2 4 2
12 9 9 0 3 0 3
12 9 9 0 3 0 4
12 9 9 0 3 0 5
12 9 9 0 3 0 6
6 6 6 4 4 0 7
3 0 0 0 3 8 8
0 0 0 0 0 0 0'''
RA = str_to_numpy_array(RA_s)

In [18]:
# import json
# children_serial = {str(k): list(v) for k, v in children.items()}
# json_str = json.dumps(children_serial)
# with open('trash/graph_activity', 'w', encoding='utf-8') as f:
#        f.write(json_str)
# Для проекта 35_0

In [19]:
gen_el = ChromosomeType( (TA, RA, 
                          init_chromosomes['lft'][0][2],  
                          init_chromosomes['lft'][0][3], 
                          init_chromosomes['lft'][0][4]))

from sampo.scheduler.genetic.operators import is_chromosome_correct

bool_0 = is_chromosome_correct(gen_el, node_indices=node_indices, parents=parents,
                     contractor_borders=contractor_borders, 
                     index2node=index2node, index2contractor=index2contractor_obj)



bool_1 = is_chromosome_correct(init_chromosomes['lft'][0], node_indices=node_indices, parents=parents,
                     contractor_borders=contractor_borders, 
                     index2node=index2node, index2contractor=index2contractor_obj)

print(bool_0, bool_1)

False True


In [ ]:
from deap import base
from sampo.schemas.time import Time
from deap.base import Toolbox
from sampo.schemas.schedule import Schedule
from sampo.api.genetic_api import ChromosomeType, FitnessFunction, Individual
toolbox = base.Toolbox()




from sampo.base import SAMPO
from sampo.schemas.time_estimator import WorkTimeEstimator, DefaultWorkEstimator
import random
work_estimator = DefaultWorkEstimator()

from sampo.scheduler.genetic.converter import (convert_schedule_to_chromosome, convert_chromosome_to_schedule,
                                               ScheduleGenerationScheme)

rand = random.Random()
SAMPO.backend.cache_scheduler_info(wg, contractors, landscape, spec, rand, work_estimator)


def register_individual_constructor(fitness_weights: tuple[int | float, ...], toolbox: base.Toolbox):
    class IndividualFitness(base.Fitness):
        weights = fitness_weights
    toolbox.register('Individual', Individual.prepare(IndividualFitness))


def evaluate(chromosome: ChromosomeType, wg: WorkGraph, toolbox: Toolbox) -> Schedule | None:
    if toolbox.validate(chromosome):
        sworks = toolbox.chromosome_to_schedule(chromosome)[0]
        return Schedule.from_scheduled_works(sworks.values(), wg)
    else:
        return None
    

toolbox.register('validate', is_chromosome_correct, node_indices=node_indices, parents=parents,
                     contractor_borders=contractor_borders, index2node=index2node, index2contractor=index2contractor_obj)
toolbox.register('chromosome_to_schedule', convert_chromosome_to_schedule, worker_pool=worker_pool,
                     index2node=index2node, index2contractor=index2contractor_obj,
                     worker_pool_indices=worker_pool_indices, assigned_parent_time=Time(0),
                     work_estimator=work_estimator, worker_name2index=worker_name2index,
                     contractor2index=contractor2index, index2zone=index2zone,
                     landscape=landscape, sgs_type=ScheduleGenerationScheme.Parallel)

toolbox.register('register_individual_constructor', register_individual_constructor, toolbox=toolbox)
toolbox.register('evaluate_chromosome', evaluate, wg=wg, toolbox=toolbox)

toolbox.register_individual_constructor((-1,))

my_pop = [ChromosomeType(init_chromosomes['lft'][0]), ChromosomeType(init_chromosomes['heft_end'][0]), gen_el]


f = TimeFitness()
f.evaluate(chromosome=my_pop[2], evaluator=toolbox.evaluate_chromosome)


(2000000000,)

In [24]:
from sampo.scheduler.genetic.operators  import SumOfResourcesPeaksFitness
f = SumOfResourcesPeaksFitness()
f.evaluate(chromosome=my_pop[2], evaluator=toolbox.evaluate_chromosome)

(2000000000,)

In [61]:
pop[2][3]

ScheduleSpec(_work2spec=defaultdict(<class 'sampo.schemas.schedule_spec.WorkSpec'>, {'53cb089e-145c-4a41-acc6-a3ce5c467896': WorkSpec(contractors=set(), assigned_workers={}, assigned_time=None, is_independent=False), '32435d5b-66d1-4955-8341-efadaaa9af6b': WorkSpec(contractors=set(), assigned_workers={}, assigned_time=None, is_independent=False), '52670047-43d8-4d29-a182-acb2b03d82ba': WorkSpec(contractors=set(), assigned_workers={}, assigned_time=None, is_independent=False), '5a78b147-e3e1-43a6-8d86-d9758b5d6f91': WorkSpec(contractors=set(), assigned_workers={}, assigned_time=None, is_independent=False), '06ef9672-5368-46d2-a59f-5e85a54d516b': WorkSpec(contractors=set(), assigned_workers={}, assigned_time=None, is_independent=False), '0e810119-6d50-4dec-ba7a-dd8bbf1ec0f1': WorkSpec(contractors=set(), assigned_workers={}, assigned_time=None, is_independent=False), '135d2878-8311-4f40-afb5-a1072a1e49c1': WorkSpec(contractors=set(), assigned_workers={}, assigned_time=None, is_independent

In [21]:
# init_chromosomes['heft_end'][0]

In [96]:
schd = convert_chromosome_to_schedule(gen_el, worker_pool,
                                   index2node,
                                   index2contractor_obj,
                                   index2zone,
                                   worker_pool_indices,
                                   worker_name2index,
                                   contractor2index,landscape)

# #schd.execution_time
from sampo.base import SAMPO
from sampo.schemas.time_estimator import WorkTimeEstimator, DefaultWorkEstimator
import random
work_estimator = DefaultWorkEstimator()
rand = random.Random
SAMPO.backend.cache_scheduler_info(wg, contractors, landscape, spec, rand, work_estimator)
fitness_f = TimeFitness()
fitness = SAMPO.backend.compute_chromosomes(fitness_f, my_pop)

# validate_schedule(schd, wg, contractors, spec)

ValueError: Unknown type of schedule generation scheme

In [23]:
schd[3]
#validate_schedule(schd, wg, contractors, spec)

[]

In [24]:
len(init_chromosomes['lft'][0])

5

In [25]:
init_popul['lft'][0].to_schedule_work_dict

{'53cb089e-145c-4a41-acc6-a3ce5c467896': ScheduledWork[work_unit=53cb089e-145c-4a41-acc6-a3ce5c467896, start_end_time=(0, 0), workers=[], contractor=c8340550-c658-8e97-b286-f5b7ea55bbef],
 '32435d5b-66d1-4955-8341-efadaaa9af6b': ScheduledWork[work_unit=32435d5b-66d1-4955-8341-efadaaa9af6b, start_end_time=(0, 1), workers=[39 driver, 19 manager, 46 handyman], contractor=630c43dd-046d-8908-08ab-819ad8118542],
 '5a78b147-e3e1-43a6-8d86-d9758b5d6f91': ScheduledWork[work_unit=5a78b147-e3e1-43a6-8d86-d9758b5d6f91, start_end_time=(1, 5), workers=[30 driver, 6 manager, 30 handyman, 6 engineer], contractor=9b793e40-436c-e982-01b4-7711bcea0501],
 '52670047-43d8-4d29-a182-acb2b03d82ba': ScheduledWork[work_unit=52670047-43d8-4d29-a182-acb2b03d82ba, start_end_time=(1, 2), workers=[42 driver, 21 manager, 57 handyman], contractor=d30f5bec-6807-6a88-56f0-84a276ff7b40],
 '199bbe9b-e9b5-473d-baf2-cf75928105ed': ScheduledWork[work_unit=199bbe9b-e9b5-473d-baf2-cf75928105ed, start_end_time=(2, 4), workers=[

In [26]:
# init_chromosomes['lft'][0][1].to_clipboard()
import pandas as pd
df = pd.DataFrame(init_chromosomes['heft_end'][0][1])
df.to_clipboard(index=False,header=False)
# i=0: Порядок выполнения задач 
# i=1: element "resources for works part of chromosome" / "assigned resources", +1 col - Stores contractors line
# i=2: element "Total resources number", см в статью, 11 Contractros, 6 Unit of resource



# order_chromosome, resource_chromosome, resource_border_chromosome, copy.deepcopy(spec), zone_changes_chromosome

In [27]:
init_chromosomes['heft_end'][0][0]

array([ 0,  1,  2,  7,  3,  9, 14, 12,  5, 15, 16, 24, 22, 18, 10, 19, 25,
       26, 27, 28, 17, 23,  6, 11,  4, 29,  8, 20, 13, 21, 30, 31])

In [28]:
init_chromosomes['lft'][0][1]

array([[  0,   0,   0,   0,   0,   0,   0],
       [ 39,   0,  46,   0,  19,   0,   1],
       [ 42,   0,  57,   0,  21,   0,   2],
       [ 30,   0,  30,   0,   6,   6,   3],
       [  7,   7,   7,   5,   5,   0,   4],
       [  6,   9,  13,   0,   2,   4,   5],
       [  6,   6,   6,   4,   4,   0,   6],
       [ 63,   0, 100,   0,  21,   0,   7],
       [  6,   6,   6,   3,   3,   0,   8],
       [  6,   8,  12,   6,   8,   0,   9],
       [ 63,  63,  63,  28,  42,   0,  10],
       [  4,   8,  12,   1,   4,   0,   0],
       [  7,  10,  13,   0,   2,   4,   1],
       [  3,   3,   3,   2,   2,   0,   2],
       [  6,   8,  12,   6,   8,   0,   3],
       [  8,   0,   9,   0,   2,   0,   4],
       [  6,   9,  12,   0,   2,   4,   5],
       [ 25,  25,  25,  17,  17,   0,   6],
       [  6,   9,  12,   4,   2,   0,   7],
       [  8,   0,   9,   0,   2,   0,   8],
       [  3,   3,   3,   2,   2,   0,   9],
       [  3,   3,   3,   2,   2,   0,  10],
       [  5,   8,  11,   0,   2,

In [29]:
contractors[1]

Contractor(id='630c43dd-046d-8908-08ab-819ad8118542', name=None, workers={'driver': 39 driver, 'fitter': 36 fitter, 'handyman': 46 handyman, 'electrician': 12 electrician, 'manager': 19 manager, 'engineer': 13 engineer}, equipments={})

In [30]:
schedule.full_schedule_df

,idx,task_id,task_name,contractor,cost,volume,start,finish,duration,workers,model_name,scheduled_work_object
0,0,53cb089e-145c-4a41-acc6-a3ce5c467896,start of project,c8340550-c658-8e97-b286-f5b7ea55bbef,0,0.000000,0,0,0,{},"{'granular_name': 'start of project', 'measure...",ScheduledWork[work_unit=53cb089e-145c-4a41-acc...
1,1,32435d5b-66d1-4955-8341-efadaaa9af6b,minimal road,c8340550-c658-8e97-b286-f5b7ea55bbef,750,7.055065,0,1,1,"{'driver': 25, 'manager': 14, 'handyman': 36}","{'granular_name': 'minimal road', 'measurement...",ScheduledWork[work_unit=32435d5b-66d1-4955-834...
2,4,5a78b147-e3e1-43a6-8d86-d9758b5d6f91,engineering preparation,9b793e40-436c-e982-01b4-7711bcea0501,3200,0.560000,1,6,5,"{'driver': 27, 'manager': 5, 'handyman': 27, '...","{'granular_name': 'engineering preparation', '...",ScheduledWork[work_unit=5a78b147-e3e1-43a6-8d8...
3,2,52670047-43d8-4d29-a182-acb2b03d82ba,temporary road,630c43dd-046d-8908-08ab-819ad8118542,910,7.055065,1,2,1,"{'driver': 34, 'manager': 16, 'handyman': 41}","{'granular_name': 'temporary road', 'measureme...",ScheduledWork[work_unit=52670047-43d8-4d29-a18...
4,3,199bbe9b-e9b5-473d-baf2-cf75928105ed,final road,d30f5bec-6807-6a88-56f0-84a276ff7b40,2100,7.055065,2,4,2,"{'driver': 37, 'manager': 18, 'handyman': 50}","{'granular_name': 'final road', 'measurement':...",ScheduledWork[work_unit=199bbe9b-e9b5-473d-baf...
5,24,06ef9672-5368-46d2-a59f-5e85a54d516b,pipe,9fc5a794-ba16-5140-4674-fd0f63b94b22,1040,0.778888,6,10,4,"{'driver': 6, 'fitter': 6, 'manager': 4, 'hand...","{'granular_name': 'pipe', 'measurement': 'km'}",ScheduledWork[work_unit=06ef9672-5368-46d2-a59...
6,6,59e9b0ff-38a0-4db8-9fce-49586013e234,node,9b793e40-436c-e982-01b4-7711bcea0501,7350,0.000000,6,27,21,"{'driver': 5, 'fitter': 7, 'manager': 7, 'hand...","{'granular_name': 'node', 'measurement': 'unit'}",ScheduledWork[work_unit=59e9b0ff-38a0-4db8-9fc...
7,27,76e5006c-aa46-4f9d-8833-60a9a67c09b2,power network,c8340550-c658-8e97-b286-f5b7ea55bbef,520,0.386063,6,10,4,"{'driver': 3, 'fitter': 3, 'manager': 2, 'hand...","{'granular_name': 'power network', 'measuremen...",ScheduledWork[work_unit=76e5006c-aa46-4f9d-883...
8,11,dadaf553-2207-4959-8317-cdde704787c2,metering installation,e6f98127-5f94-0c18-252b-06c4af20baff,14000,0.000000,6,56,50,"{'driver': 5, 'fitter': 8, 'manager': 1, 'hand...","{'granular_name': 'metering installation', 'me...",ScheduledWork[work_unit=dadaf553-2207-4959-831...
9,21,d325bd2d-b511-4653-b21e-41d25cea08be,power line,7f92f7b8-ea2e-5f7b-b4c2-5697e8ba4a45,960,2.762447,6,7,1,"{'driver': 22, 'fitter': 22, 'manager': 15, 'h...","{'granular_name': 'power line', 'measurement':...",ScheduledWork[work_unit=d325bd2d-b511-4653-b21...
